In [ ]:
import openai
import torch
import asyncio
import functools

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from assistance import _ctx
from assistance._config import load_faq_data
from assistance._keys import get_openai_api_key
from assistance._openai import get_embedding

In [ ]:
_ctx.open_session()
OPEN_AI_API_KEY = get_openai_api_key()

In [ ]:
faq_data = await load_faq_data('jims-ac')

In [ ]:
question = faq_data['items'][0]["question"]
question

In [ ]:
all_questions = tuple([item['question'] for item in faq_data['items']])
# all_questions

In [ ]:
# embeddings = await asyncio.gather(*[_get_embedding(question) for question in all_questions])

In [ ]:
# embeddings

In [ ]:
async def _get_cuda_embedding(question: str) -> torch.Tensor:
    embedding = await get_embedding(block=question, api_key=OPEN_AI_API_KEY)
    return torch.tensor([embedding], device="cuda")

In [ ]:
async def _get_cuda_embeddings(questions: str) -> torch.Tensor:
    embeddings = await asyncio.gather(*[get_embedding(block=question, api_key=OPEN_AI_API_KEY) for question in all_questions])
    return torch.tensor(embeddings, device="cuda")

In [ ]:
embeddings = await _get_cuda_embeddings(all_questions)

In [ ]:
embeddings

In [ ]:
embeddings[0, :]

In [ ]:
query = await _get_cuda_embedding("What sort of age limits are there?")

In [ ]:
query.shape

In [ ]:
@torch.jit.script
def _top_k_embeddings(query, embeddings, k):
    transpose_query = query.T
    embeddings_norm = torch.linalg.norm(embeddings, dim=1, keepdim=True)
    query_norm = torch.linalg.norm(transpose_query, dim=0, keepdim=True)
    
    cosine_similarity = ((embeddings @ transpose_query) / (embeddings_norm @ query_norm)).T
    
    return torch.topk(cosine_similarity, k)

In [ ]:
def top_k_embeddings(query, embeddings, k):
    k = torch.tensor(k, device="cuda")
    cosine_similarity, index = _top_k_embeddings(query, embeddings, k)
    
    index = index.tolist()
    cosine_similarity = cosine_similarity.tolist()
    
    assert len(index) == 1
    assert len(cosine_similarity) == 1
    
    return index[0], cosine_similarity[0]

In [ ]:
indices, scores = top_k_embeddings(query, embeddings, 3)

In [ ]:
scores

In [ ]:
indices

In [ ]:
[all_questions[i] for i in indices]

In [ ]:
async def get_top_questions_and_answers(faq_data, embeddings, query, k=3):
    query_embedding = await _get_cuda_embedding(query)
    
    indices, scores = top_k_embeddings(query_embedding, embeddings, k)
    
    return [faq_data['items'][i] for i in indices]   

In [ ]:
query = "What if I have turned 50? Is there any issues there?"
await get_top_questions_and_answers(faq_data, embeddings, query)

In [ ]:
list(query)

In [ ]:
embeddings.shape

In [ ]:
query.size()

In [ ]:
Z = torch.rand(10, 4)
B = Z[0:4].T

In [ ]:
Z.size()

In [ ]:
Z_norm = torch.linalg.norm(Z, dim=1, keepdim=True)  # Size (n, 1).
Z_norm.size()

In [ ]:
# result['data'][0]['embedding']

In [ ]:
Z = torch.rand(10, 4)
B = Z[0:4].T

Z_norm = torch.linalg.norm(Z, dim=1, keepdim=True)  # Size (n, 1).
B_norm = torch.linalg.norm(B, dim=0, keepdim=True)  # Size (1, b).

# Distance matrix of size (b, n).
cosine_similarity = ((Z @ B) / (Z_norm @ B_norm)).T
cosine_distance = 1 - cosine_similarity

In [ ]:
cosine_similarity

In [ ]:
B

In [ ]:
cosine_distance